In [1]:
import numpy as np
from collections import OrderedDict as odict
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [5]:
arch = {'knl':(cm.Greens,450,0.5,0.33),'skl':(cm.Greys,200,0.5,0.75),'p100':(cm.Blues,550,0.5,0.43),
        'v100':(cm.Purples,850,0.5,0.85),'i5':(cm.Wistia,30,0.5,0.79),'gtx1060':(cm.Oranges,155,0.5,0.70)}
intens={1:0.8, 2:0.6, 4:0.4}
marker={2:'d', 3:'o', 4:'s',5:'p'}
#(hardware name, number of nodes)
filesD = {
         'knl_mpi1':('knl',1), 'knl_mpi2':('knl',2), 'knl_mpi4':('knl',4),
         'skl_mpi1':('skl',1), 'skl_mpi2':('skl',2), 'skl_mpi4':('skl',4),
         'i5':('i5',1),
         'p100_mpi1':('p100',1), 'p100_mpi2':('p100',2), 'p100_mpi4':('p100',4),
         'v100_mpi1':('v100',1), 'v100_mpi2':('v100',2), 'v100_mpi4':('v100',4),
         'gtx1060':('gtx1060',1)
        }

files = odict(sorted(filesD.items(), key= lambda t : t[1][1]))
pd.set_option('precision',1)

In [15]:
#(axpby,dot,dxdy)
types = {}
types['scal'] = (1,0,0)
types['axpby'] = (1,0,0)
types['pointwiseDot'] = (1,0,0)
types['dot'] = (0,1,0)
types['dx'] = (0,0,1)
types['dy'] = (0,0,1)
types['arakawa'] = (3,0,6)
types['cg'] = (6,2,6)
memops = {}
memops['scal']= (2,0,0)
memops['axpby']= (3,0,0)
memops['pointwiseDot']= (6,0,0)
memops['dot']= (0,2,0)
memops['dx']= (0,0,3)
memops['dy']= (0,0,3)
memops['arakawa'] = (16,0,18) # two aliases in pointwiseDot
memops['cg'] = (20,4,18)

In [16]:
theo = pd.read_csv('performance.csv',delimiter=' ')
theo.set_index('arch',inplace=True)
theo.index.name = None
#theo

In [29]:
#for f, v in files.items() :#{'knl_mpi2':('knl',2)}.items():
f ='knl_mpi2'
v = ('knl',2)
df=pd.read_csv('benchmark_'+f+'.csv', delimiter=' ')
#add size and get rid of non-relevant columns
df.insert(0,'size', 8*df['n']*df['n']*df['Nx']*df['Ny']/1e6/v[1])
dfr = df[['n','Nx','Ny','size']+list(memops.keys())]
#compute mean and standard derivation of 'same' groups 
dfr=dfr.groupby(['n', 'Nx','Ny','size']).mean()
dfr=dfr.reset_index(level=['n','Nx','Ny','size'])
dfr['FirstLevel']='meas'

dfr.columns=pd.MultiIndex.from_product([dfr.columns,['meas']])
dfr

,n,Nx,Ny,size,arakawa,cg,scal,dot,dx,pointwiseDot,axpby,dy,FirstLevel
,meas,meas,meas,meas,meas,meas,meas,meas,meas,meas,meas,meas,meas
0,2,128,128,0.3,5.0e-04,8.4e-04,3.3e-05,9.4e-05,3.5e-04,8.6e-05,1.9e-04,1.3e-04,meas
1,2,256,256,1.0,6.6e-04,9.1e-04,1.8e-05,1.1e-04,3.5e-05,1.4e-05,1.5e-05,1.5e-04,meas
2,2,384,384,2.4,9.3e-04,1.3e-03,2.2e-05,1.3e-04,4.7e-05,1.9e-05,1.7e-05,1.7e-04,meas
3,2,512,512,4.2,1.4e-03,1.8e-03,2.4e-05,1.6e-04,7.1e-05,2.6e-05,1.9e-05,2.2e-04,meas
4,2,768,768,9.4,2.3e-03,2.9e-03,2.4e-05,2.5e-04,1.2e-04,1.4e-04,3.3e-05,3.5e-04,meas
5,2,1024,1024,16.8,3.6e-03,4.3e-03,3.3e-05,4.4e-04,2.3e-04,2.9e-04,7.0e-05,5.5e-04,meas
6,2,1536,1536,37.7,5.9e-03,7.3e-03,1.4e-04,8.1e-04,5.0e-04,5.4e-04,2.7e-04,8.4e-04,meas
7,2,2048,2048,67.1,9.5e-03,1.2e-02,3.7e-04,1.2e-03,8.4e-04,9.7e-04,4.6e-04,1.3e-03,meas
8,3,128,128,0.6,6.3e-04,8.5e-04,1.3e-05,9.9e-05,4.0e-04,1.4e-05,1.7e-05,1.5e-04,meas
